In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [27]:
# Load covariance matrix
cov_data = np.loadtxt("lsst_y1_cov")
# Parsing cov data
dv_length = 1560//2
cov = np.zeros((dv_length, dv_length))
for line in cov_data:
    i = int(line[0])
    j = int(line[1])
    if i < dv_length and j < dv_length:
        cov[i,j] = line[8] + line[9]
        cov[j,i] = cov[i,j]
invcov = np.linalg.inv(cov)
# Loading angular scale cuts
masks = {}
for i in range(1, 7):
    masks[i]= np.loadtxt(f"lsst_y1_M{i}_GGLOLAP0.05.mask", unpack=True, usecols=(1,), )[:dv_length]

In [6]:
def compute_delta_chi2(cosmo="desi2cmbpanplus", mask=None):
    cola_dv = np.loadtxt(f"cola_{cosmo}.modelvector", usecols=1, max_rows=780)
    ee2_dv = np.loadtxt(f"ee2_{cosmo}.modelvector", usecols=1, max_rows=780)
    diff = cola_dv - ee2_dv
    # Compute chi-squared
    if mask is not None: diff = diff*mask
    chi2 = np.dot(diff, np.dot(invcov, diff))
    print(f"Cosmology {cosmo}: \Delta \chi^2 = {chi2}")

In [7]:
compute_delta_chi2("ee2ref")
compute_delta_chi2("desi2cmbpanplus")

Cosmology ee2ref: \Delta \chi^2 = 0.0006632988021168517
Cosmology desi2cmbpanplus: \Delta \chi^2 = 3.279578382006987


Lesson: at EE2REF cosmology, $\Delta\chi^2 = 6\times 10^{-4}$. We need to focus on cosmologies away from LCDM!

In [60]:
for param in ["Omega_m", "As", "ns"]:
    for param_variation in ["hi", "low"]:
        compute_delta_chi2(f"{param}{param_variation}_whi_walow",  mask=masks[3])
        compute_delta_chi2(f"{param}{param_variation}_wlow_wahi",  mask=masks[3])
        compute_delta_chi2(f"{param}{param_variation}_whi_wahi",   mask=masks[3])
        compute_delta_chi2(f"{param}{param_variation}_wlow_walow", mask=masks[3])
        print("-----")

Cosmology Omega_mhi_whi_walow: \Delta \chi^2 = 0.05436969535438241
Cosmology Omega_mhi_wlow_wahi: \Delta \chi^2 = 0.22727838256242722
Cosmology Omega_mhi_whi_wahi: \Delta \chi^2 = 0.2850700654358556
Cosmology Omega_mhi_wlow_walow: \Delta \chi^2 = 1.7386956651423753
-----
Cosmology Omega_mlow_whi_walow: \Delta \chi^2 = 0.05207104182883727
Cosmology Omega_mlow_wlow_wahi: \Delta \chi^2 = 0.03145366740101016
Cosmology Omega_mlow_whi_wahi: \Delta \chi^2 = 0.9796903081134208
Cosmology Omega_mlow_wlow_walow: \Delta \chi^2 = 0.23596001051025342
-----
Cosmology Ashi_whi_walow: \Delta \chi^2 = 0.048417490862251215
Cosmology Ashi_wlow_wahi: \Delta \chi^2 = 0.06723531417252787
Cosmology Ashi_whi_wahi: \Delta \chi^2 = 0.6182426475500158
Cosmology Ashi_wlow_walow: \Delta \chi^2 = 0.7149758766430461
-----
Cosmology Aslow_whi_walow: \Delta \chi^2 = 0.017114255712461494
Cosmology Aslow_wlow_wahi: \Delta \chi^2 = 0.019527862771323984
Cosmology Aslow_whi_wahi: \Delta \chi^2 = 0.26159373256426643
Cosmolog